In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
#from mmseg.registry import MODELS
import os
import matplotlib.image as mpimage
import torchvision.utils as vutils
import matplotlib.colors as c
from torch import tensor
import sys
# helper methods for preprocessing
def calculate_weight(channels, depth, single_color, color_opponency, black_white):
    weight_array = np.ones((channels, depth * 3, 1, 1))
    print("depth" + str(depth))
    if depth == 1:

        print("sc")
        print(single_color)
        print("bw")
        print(black_white)
        
        if not single_color and not color_opponency and not black_white: # Average black-white image
            weight_array[:, :3, :, :] *= 1 / 3
        elif color_opponency and not single_color and not black_white: #color opponency depth 0
            print("color_opponency")
            copy = weight_array[0, :, :, :]
            weight_array = np.zeros((channels, depth * 3, 1, 1))
            weight_array[0, :, :, :] = copy
            base_r_g = [0.5, -0.5, 0]
            #r_g_value = [-(1 / ((depth - 1) * 2)), (1 / ((depth - 1) * 2)), 0]
            base_b_y = [-0.5 / 3, -0.5 / 3, 1 / 3]
            #b_y_value = [(0.5 / (depth * 3 - 3)), (0.5 / (depth * 3 - 3)), -(1 / (depth * 3 - 3))]

            for i in range(depth * 3):
                if i < 3:
                    # Setze die ersten drei Werte direkt auf 0.5, -0.5, 0
                    weight_array[1, i, 0, 0] = base_r_g[i]
                    weight_array[2, i, 0, 0] = base_b_y[i]
                #else:
                    # Fülle den Rest mit dem Pattern entsprechend depth
                #    index = (i - 3) % 3
                #    weight_array[1, i, 0, 0] = r_g_value[index]
                #    weight_array[2, i, 0, 0] = b_y_value[index]
            weight_array[0,:,:,:] = 1/3

            
            print("HERE")
            #for i in range(1,3):
             #   weight_array[1, i, 0, 0] = [0.5, -0.5, 0][i]
              #  weight_array[2, i, 0, 0] = [-0.5 / 3, -0.5 / 3, 1 / 3][i]
        elif black_white and not color_opponency and not single_color: # Luminance black-white image
            print("black_white")
            weight_array[:, 0, :, :] *= 0.299
            weight_array[:, 1, :, :] *= 0.587
            weight_array[:, 2, :, :] *= 0.114
    else:
        weight_array[:, :3, :, :] *= 1 / 3
        weight_array[:, 3:, :, :] *= -(1 / (depth * 3 - 3))

        if channels == 3 and single_color:
            print("single_color")
            for c in range(channels):
                weight_array[c, :, 0, 0] = 0  # Setze alles auf 0
                weight_array[c, c, 0, 0] = 1  # Setze die 1 an die richtige Stelle

                for i in range(1, depth):
                    weight_array[c, i * 3 + c, 0, 0] = -1 / (depth - 1)

        elif channels == 3 and color_opponency:
            print("color_opponency")
            copy = weight_array[0, :, :, :]
            weight_array = np.zeros((channels, depth * 3, 1, 1))
            weight_array[0, :, :, :] = copy
            base_r_g = [0.5, -0.5, 0]
            r_g_value = [-(1 / ((depth - 1) * 2)), (1 / ((depth - 1) * 2)), 0]
            base_b_y = [-0.5 / 3, -0.5 / 3, 1 / 3]
            b_y_value = [(0.5 / (depth * 3 - 3)), (0.5 / (depth * 3 - 3)), -(1 / (depth * 3 - 3))]

            for i in range(depth * 3):
                if i < 3:
                    # Setze die ersten drei Werte direkt auf 0.5, -0.5, 0
                    weight_array[1, i, 0, 0] = base_r_g[i]
                    weight_array[2, i, 0, 0] = base_b_y[i]
                else:
                    # Fülle den Rest mit dem Pattern entsprechend depth
                    index = (i - 3) % 3
                    weight_array[1, i, 0, 0] = r_g_value[index]
                    weight_array[2, i, 0, 0] = b_y_value[index]


    print(weight_array)
    return weight_array

def create_blur_kernel():
    kernel = np.zeros((3, 3))

    for i in range(3):
        for j in range(3):
            kernel[i, j] = 1 / 9

    return kernel

def save_image(image_tensor, where, save_name, channels, path, training, sc, co, orig):

    if training:
        savepath = path + "/images"
    else:
        savepath = path + "/images_test"

    savepath = path + "/new_images"
    if sc:
        savepath = savepath + "/sc"
    if orig:    
        savepath = savepath + "/orig"
    if co:
        savepath = savepath + "/co"
    if (not sc and not co and not orig):
        savepath = savepath + "/bw"


    os.makedirs(savepath, exist_ok=True)
    
    image_name = save_name + "_" + where
    # Path(path + "/" + savepath +"/" + save_name).mkdir(parents=True, exist_ok=True)

    if sc:
        cmap_R = c.LinearSegmentedColormap.from_list("cmap_R",
                                                 ['black', 'white', '#f00'])  # hier sind die colormaps definiert,
        cmap_G = c.LinearSegmentedColormap.from_list("cmap_G", ['black', 'white',
                                                            '#0f0'])  # in denen die channel angezeigt werden
        cmap_B = c.LinearSegmentedColormap.from_list("cmap_B", ['black', 'white', '#00f'])
    elif co:
        cmap_R = c.LinearSegmentedColormap.from_list("cmap_R",
                                                 ['black', 'white', 'black'])  # hier sind die colormaps definiert,
        cmap_G = c.LinearSegmentedColormap.from_list("cmap_G", ['#0f0', 'white',
                                                            '#f00'])  # in denen die channel angezeigt werden
        cmap_B = c.LinearSegmentedColormap.from_list("cmap_B", ['#ff0', 'white', '#00f'])   
    elif orig:
        cmap_R = c.LinearSegmentedColormap.from_list("cmap_R",
                                                 ['white', '#f00'])  # hier sind die colormaps definiert,
        cmap_G = c.LinearSegmentedColormap.from_list("cmap_G", ['white',
                                                            '#0f0'])  # in denen die channel angezeigt werden
        cmap_B = c.LinearSegmentedColormap.from_list("cmap_B", ['white', '#00f'])
    else:
        cmap_R = c.LinearSegmentedColormap.from_list("cmap_R",
                                                 ['black', 'white', 'black'])  # hier sind die colormaps definiert,
        cmap_G = c.LinearSegmentedColormap.from_list("cmap_G", ['black', 'white',
                                                            'black'])  # in denen die channel angezeigt werden
        cmap_B = c.LinearSegmentedColormap.from_list("cmap_B", ['black', 'white', 'black'])
        
    boundary_red = max(torch.max(image_tensor[0]), -torch.min(image_tensor[0]))
    #f, axarr = plt.subplots(5, 4, figsize=(20, 20))
    print("Red min:")
    print(torch.min(image_tensor[0]))
    print("Red max:")
    print(torch.max(image_tensor[0]))
    img_norm = (image_tensor - torch.min(image_tensor)) / (torch.max(image_tensor) - torch.min(image_tensor))
    # img_norm = image_tensor / torch.max(image_tensor)
    #axarr[0][0].imshow(img_norm.detach().cpu().permute(1, 2, 0))
    #axarr[0][1].imshow(image_tensor[0].detach().cpu(), cmap=cmap_R)
    #vutils.save_image(img_norm, "" + savepath + "/" + image_name + "RGB.pdf")
    #plt.imsave("" + savepath + "/" + image_name + "-r.png", image_tensor[0].detach().cpu(), cmap=cmap_R,
                   #vmin=-boundary_red, vmax=boundary_red)
     #               vmin=0, vmax=1)

    if channels > 1:
        boundary_green = max(torch.max(image_tensor[1]), -torch.min(image_tensor[1]))
        boundary_blue = max(torch.max(image_tensor[2]), -torch.min(image_tensor[2]))
        #axarr[0][2].imshow(image_tensor[1].detach().cpu(), cmap=cmap_G)
        #axarr[0][3].imshow(image_tensor[2].detach().cpu(), cmap=cmap_B)
        #plt.imsave("" + savepath + "/" + image_name + "-g.png", image_tensor[1].detach().cpu(), cmap=cmap_G,
                       #vmin=-boundary_green, vmax=boundary_green)
         #               vmin=0, vmax=1)
        plt.imsave("" + savepath + "/" + image_name + "-b.png", image_tensor[2].detach().cpu(), cmap=cmap_B,
                       vmin=-boundary_blue, vmax=boundary_blue)
                       # vmin=0, vmax=1)

    #plt.close()
#@MODELS.register_module()
class BlurPreprocessing(nn.Module):
    def __init__(self, blur_bool, blur_depth, single_color, color_opponency, channels, path, training, black_white):
        super().__init__()
        self.blur = blur_bool
        self.num_images = blur_depth + 1
        self.single_color = single_color
        self.color_opponency = color_opponency
        self.channels = channels
        self.write = True
        self.path = path
        self.training = training
        self.black_white = black_white

        #self.maxpool = nn.MaxPool2d(2, 2)
        
        if self.blur:
            blur_kernel = create_blur_kernel()
            self.conv_blur = nn.Conv2d(
                3, 3 * self.num_images, 3, stride=1, padding=1, groups=3, bias=False
            )

            self.conv_blur.weight = nn.Parameter(
                tensor(np.array([[blur_kernel],
                                 [blur_kernel],
                                 [blur_kernel]]),
                       requires_grad=False).float()
            )

            for param in self.conv_blur.parameters():
                param.requires_grad = False

            self.custom_layer = nn.Conv2d(
                self.num_images * 3, out_channels=channels, kernel_size=1,
                stride=1, padding=0, bias=False
            )

            weight_array = calculate_weight(
                self.channels, self.num_images, self.single_color,
                self.color_opponency, self.black_white
            )
            self.custom_layer.weight = nn.Parameter(
                tensor(np.array(weight_array), requires_grad=True).float()
            )

            # freeze preprocessing
            for param in self.custom_layer.parameters():
                param.requires_grad = False

            self.change_channel_layer = nn.Conv2d(
                in_channels=1, out_channels=3, kernel_size=1, stride=1, padding=0
            )

            print("preprocessing initialized")

    def forward(self, x, save_name="image"):
        #save_image(x[0], "orig", save_name, self.channels, self.path, self.training, self.single_color, self.color_opponency, True)
        if self.blur:
            if False:#self.write:
                print("saving image before preprocessing")
                save_image(x[0], "before", save_name, self.channels, self.path, self.training)
                np.save(f"{save_name}_before.npy", x[0].detach().cpu().numpy())

            concat_image = x
            for i in range(self.num_images - 1):
                x = self.conv_blur(x)
                concat_image = torch.concat([concat_image, x], dim=1)

            x = self.custom_layer(concat_image)

            if self.channels == 1:
                x = self.change_channel_layer(x)

        if self.write:
            print("saving image after preprocessing")
            print(self.path)
            print(self.channels)
            print(save_name)
            #pooled = self.maxpool(x[0])
            save_image(x[0], "after", save_name, self.channels, self.path, self.training, self.single_color, self.color_opponency, False)

            #save_image(pooled, "processed-pooled", save_name, self.channels, self.path, self.training, self.single_color, self.color_opponency, False)


            
            # save a copy as png (normalized)
            output_img = x[0].detach().cpu().numpy()
            
            #output_img = np.transpose(output_img, (1, 2, 0))  # (H, W, C)

            boundary = abs(output_img).max()
           # normalize to 0-1 with 0->0.5
            output_img = (output_img + boundary) / (boundary + boundary + 1e-8)
            
            # clip between 0.4 and 0.6
            output_img = np.clip(output_img, 0.47, 0.53)
            
            # rescale clipped range to 0-1
            output_img = (output_img - 0.47) / (0.53 - 0.47)
            output_tensor = torch.tensor(output_img)
            
            #save_image(output_tensor, "sig", save_name, self.channels, self.path, self.training, self.single_color, self.color_opponency, False)
            #imageio.imwrite(f"{save_name}_after.png", output_img)
            # convert to 0-255 and uint8
            output_img = (output_img * 255).astype(np.uint8)
            
            #norm_img = np.sqrt(norm_img)*255
            #output_img = np.clip(norm_img*255, 0, 255).astype(np.uint8)
            #imageio.imwrite(f"{save_name}_after.pdf", output_img)
            
            # disable further writing
            #self.write = False

        return x


: 

In [4]:
import glob
import os
from PIL import Image
import torchvision.transforms as T
import torch

## Template

In [ ]:
import glob
import os
from PIL import Image
import torchvision.transforms as T
import torch

blur_module = BlurPreprocessing(
    blur_bool=False,
    blur_depth=0,
    single_color=False,
    color_opponency=False,
    channels=3,   # or 3
    path=".",
    training=False,
    black_white=True
)

# Transform for images
transform = T.Compose([
    T.ToTensor()
])

# Get all images in ./images
image_files = glob.glob("./slices/1/*.*")

for img_path in image_files:
    base = os.path.splitext(os.path.basename(img_path))[0]
    print(f"\nProcessing {base} ...")

    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0)  # (1,3,H,W)

    with torch.no_grad():
        out = blur_module(x, save_name=base)


Processing image_beforeRGB-1 ...
saving image after preprocessing

Processing image_before_acdc_fog_RGB-1 ...
saving image after preprocessing

Processing image_before_acdc_night_RGB-1 ...
saving image after preprocessing

Processing image_before_acdc_rain_RGB-1 ...
saving image after preprocessing

Processing image_before_acdc_snow_RGB-1 ...
saving image after preprocessing

Processing image_before_dark_zurich_RGB-1 ...
saving image after preprocessing


## Grayscale

In [52]:


blur_module = BlurPreprocessing(
    blur_bool=True,
    blur_depth=5,
    single_color=False,
    color_opponency=False,
    channels=3,   # or 3
    path=".",
    training=False,
    black_white=True
)

# Transform for images
transform = T.Compose([
    T.ToTensor()
])

# Get all images in ./images
image_files = glob.glob("./slices/1/*.*")

for img_path in image_files:
    base = os.path.splitext(os.path.basename(img_path))[0]
    base = base + "gray"
    print(f"\nProcessing {base} ...")

    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0)  # (1,3,H,W)

    with torch.no_grad():
        out = blur_module(x, save_name=base)

depth6
[[[[ 0.33333333]]

  [[ 0.33333333]]

  [[ 0.33333333]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]]


 [[[ 0.33333333]]

  [[ 0.33333333]]

  [[ 0.33333333]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]]


 [[[ 0.33333333]]

  [[ 0.33333333]]

  [[ 0.33333333]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

 

## Single Color

In [53]:
blur_module = BlurPreprocessing(
    blur_bool=True,
    blur_depth=5,
    single_color=True,
    color_opponency=False,
    channels=3,   # or 3
    path=".",
    training=False,
    black_white=True
)

# Transform for images
transform = T.Compose([
    T.ToTensor()
])

# Get all images in ./images
image_files = glob.glob("./slices/1/*.*")

for img_path in image_files:
    base = os.path.splitext(os.path.basename(img_path))[0]
    base = base + "sc"
    print(f"\nProcessing {base} ...")

    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0)  # (1,3,H,W)

    with torch.no_grad():
        out = blur_module(x, save_name=base)

depth6
single_color
[[[[ 1. ]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]

  [[ 0. ]]

  [[ 0. ]]]


 [[[ 0. ]]

  [[ 1. ]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]

  [[ 0. ]]]


 [[[ 0. ]]

  [[ 0. ]]

  [[ 1. ]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]

  [[ 0. ]]

  [[ 0. ]]

  [[-0.2]]]]
preprocessing initialized

Processing image_beforeRGB-1_-1sc ...
saving image after preprocessing
.
3
image_beforeRGB-1_-1sc
Red min:
tensor(0.)
Red max:
tensor(1.0000)

Processing image_before_acdc_fog_RGB-1_-1sc ...
saving image after preprocessing
.
3
image_before_acdc_fog_RGB-1_-1sc
Red min:
tensor(0.)
Red max:
tensor(1.0

## Color opponency

In [7]:
blur_module = BlurPreprocessing(
    blur_bool=True,
    blur_depth=5,
    single_color=False,
    color_opponency=True,
    channels=3,   # or 3
    path=".",
    training=False,
    black_white=True
)

# Transform for images
transform = T.Compose([
    T.ToTensor()
])

# Get all images in ./images
image_files = glob.glob("./slices/1/*.*")

for img_path in image_files:
    base = os.path.splitext(os.path.basename(img_path))[0]
    base = base + "co"
    print(f"\nProcessing {base} ...")

    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0)  # (1,3,H,W)

    with torch.no_grad():
        out = blur_module(x, save_name=base)

depth6
color_opponency
[[[[ 0.33333333]]

  [[ 0.33333333]]

  [[ 0.33333333]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]

  [[-0.06666667]]]


 [[[ 0.5       ]]

  [[-0.5       ]]

  [[ 0.        ]]

  [[-0.1       ]]

  [[ 0.1       ]]

  [[ 0.        ]]

  [[-0.1       ]]

  [[ 0.1       ]]

  [[ 0.        ]]

  [[-0.1       ]]

  [[ 0.1       ]]

  [[ 0.        ]]

  [[-0.1       ]]

  [[ 0.1       ]]

  [[ 0.        ]]

  [[-0.1       ]]

  [[ 0.1       ]]

  [[ 0.        ]]]


 [[[-0.16666667]]

  [[-0.16666667]]

  [[ 0.33333333]]

  [[ 0.03333333]]

  [[ 0.03333333]]

  [[-0.06666667]]

  [[ 0.03333333]]

  [[ 0.03333333]]

  [[-0.06666667]]

  [[ 0.03333333]]

  [[ 0.03333333]]

  [[-0.06666667]]

  [[ 0.03333333]]

  [[ 0.03333333]]

  [[-0.06666667]]

  [[

## CO D0

In [8]:
blur_module = BlurPreprocessing(
    blur_bool=True,
    blur_depth=0,
    single_color=False,
    color_opponency=True,
    channels=3,   # or 3
    path=".",
    training=False,
    black_white=False
)

# Transform for images
transform = T.Compose([
    T.ToTensor()
])

# Get all images in ./images
image_files = glob.glob("./slices/3/*.*")

for img_path in image_files:
    base = os.path.splitext(os.path.basename(img_path))[0]
    base = base + "co_d0"
    print(f"\nProcessing {base} ...")

    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0)  # (1,3,H,W)

    with torch.no_grad():
        out = blur_module(x, save_name=base)

depth1
sc
False
bw
False
color_opponency
HERE
[[[[ 0.33333333]]

  [[ 0.33333333]]

  [[ 0.33333333]]]


 [[[ 0.5       ]]

  [[-0.5       ]]

  [[ 0.        ]]]


 [[[-0.16666667]]

  [[-0.16666667]]

  [[ 0.33333333]]]]
preprocessing initialized

Processing image_beforeRGB-1_-3co_d0 ...
saving image after preprocessing
.
3
image_beforeRGB-1_-3co_d0
Red min:
tensor(0.0824)
Red max:
tensor(0.9608)

Processing image_before_acdc_fog_RGB-1_-3co_d0 ...
saving image after preprocessing
.
3
image_before_acdc_fog_RGB-1_-3co_d0
Red min:
tensor(0.0850)
Red max:
tensor(0.8601)

Processing image_before_acdc_night_RGB-1_-3co_d0 ...
saving image after preprocessing
.
3
image_before_acdc_night_RGB-1_-3co_d0
Red min:
tensor(0.0275)
Red max:
tensor(0.9281)

Processing image_before_acdc_rain_RGB-1_-3co_d0 ...
saving image after preprocessing
.
3
image_before_acdc_rain_RGB-1_-3co_d0
Red min:
tensor(0.0863)
Red max:
tensor(0.8915)

Processing image_before_acdc_snow_RGB-1_-3co_d0 ...
saving image after pr

In [19]:
from PIL import Image, ImageOps

# Open the PNG
img = Image.open("new_images/co crop/image_beforeRGB-1co_d0_after-r.png")

# Invert (if it has alpha channel, handle it separately)
if img.mode == 'RGBA':
    r, g, b, a = img.split()
    rgb = Image.merge("RGB", (r, g, b))
    inverted_rgb = ImageOps.invert(rgb)
    inverted = Image.merge("RGBA", (*inverted_rgb.split(), a))
else:
    inverted = ImageOps.invert(img)

# Save result
inverted.save("new_images/co crop/image_beforeRGB-1co_d0_after-r-i.png")


In [20]:
import os
from PIL import Image

# Input and output directories
input_dir = "images"      # your folder with input images
output_dir = "slices"     # where to save results
os.makedirs(output_dir, exist_ok=True)

# Loop over files
for filename in os.listdir(input_dir):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):
        filepath = os.path.join(input_dir, filename)
        img = Image.open(filepath)

        width, height = img.size
        slice_width = width // 3  # cut into 3 equal vertical parts

        # Remove extension for naming
        base, ext = os.path.splitext(filename)

        for i in range(3):
            left = i * slice_width
            right = (i + 1) * slice_width
            box = (left, 0, right, height)
            slice_img = img.crop(box)

            outname = f"{base}_-{i+1}{ext}"
            outpath = os.path.join(output_dir, outname)
            slice_img.save(outpath)

            print(f"Saved {outpath} ({slice_img.size})")


Saved slices\image_beforeRGB-1_-1.png ((1896, 2845))
Saved slices\image_beforeRGB-1_-2.png ((1896, 2845))
Saved slices\image_beforeRGB-1_-3.png ((1896, 2845))
Saved slices\image_before_acdc_fog_RGB-1_-1.png ((1778, 3000))
Saved slices\image_before_acdc_fog_RGB-1_-2.png ((1778, 3000))
Saved slices\image_before_acdc_fog_RGB-1_-3.png ((1778, 3000))
Saved slices\image_before_acdc_night_RGB-1_-1.png ((1778, 3000))
Saved slices\image_before_acdc_night_RGB-1_-2.png ((1778, 3000))
Saved slices\image_before_acdc_night_RGB-1_-3.png ((1778, 3000))
Saved slices\image_before_acdc_rain_RGB-1_-1.png ((1778, 3000))
Saved slices\image_before_acdc_rain_RGB-1_-2.png ((1778, 3000))
Saved slices\image_before_acdc_rain_RGB-1_-3.png ((1778, 3000))
Saved slices\image_before_acdc_snow_RGB-1_-1.png ((1778, 3000))
Saved slices\image_before_acdc_snow_RGB-1_-2.png ((1778, 3000))
Saved slices\image_before_acdc_snow_RGB-1_-3.png ((1778, 3000))
Saved slices\image_before_dark_zurich_RGB-1_-1.png ((1778, 3000))
Saved s